<a href="https://colab.research.google.com/github/poppingary/name-entity-recognition/blob/main/ExtendedCode/tagLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import package

In [ ]:
!pip install --upgrade pip

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [ ]:
!pip install allennlp==2.1.0 allennlp-models==2.1.0

     |████████████████████████████████| 585 kB 5.1 MB/s            
     |████████████████████████████████| 407 kB 52.7 MB/s            
     |████████████████████████████████| 776.8 MB 15 kB/s             
     |████████████████████████████████| 1.9 MB 34.2 MB/s            
     |████████████████████████████████| 259 kB 60.5 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 12.8 MB 16.2 MB/s            
     |████████████████████████████████| 1.2 MB 36.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 124 kB 63.8 MB/s            
     |████████████████████████████████| 131 kB 51.8 MB/s            
     |████████████████████████████████| 64 kB 2.7 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 56 kB 3.6 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 79 kB 6.5 MB/s             
     |████

In [ ]:
from tqdm import tqdm

## import predictor

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-elmo.2021-02-12.tar.gz")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'
downloading: 100%|##########| 374434792/374434792 [00:12<00:00, 29771203.32B/s]
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


## generate sentences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hard_temp_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/SentenceTemplate/hard_sentence_templates.txt'
easy_temp_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/SentenceTemplate/easy_sentence_templates.txt'
hard_name_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/NameSample/rare_names.txt'
easy_name_fp = '/content/drive/MyDrive/IntroToMachineLearning/SentenceSample/NameSample/common_names.txt'

In [ ]:
def read_temp(fp):
  all_string = ""
  with open(fp, 'r', encoding='utf-8-sig') as f:
    for l in f:
      all_string += l
  sents = all_string.split(".")
  return [s.strip()+'.' for s in sents if s != '']

In [ ]:
def read_name(fp):
  names = []
  with open(fp, 'r', encoding='utf-8-sig') as f:
    for l in f:
      names.append(l.strip())
  return names

In [ ]:
easy_temps = read_temp(easy_temp_fp)
hard_temps = read_temp(hard_temp_fp)

In [ ]:
easy_names = read_name(easy_name_fp)
hard_names = read_name(hard_name_fp)

In [ ]:
def predict_sent(temps, names):

  def is_correct(tags, words, name):
    for tag, word in zip(tags, words):
      if word == name and tag == 'U-PER':
        return True
    return False

  correct = [] #[(sent, name, pred), ...]
  wrong = []
  for temp in tqdm(temps):
    for name in names:
      sent = temp.replace('*', name)
      pred = predictor.predict(sent)
      if is_correct(pred['tags'], pred['words'], name):
        correct.append((sent, name, pred))
      else:
        wrong.append((sent, name, pred))
  print("")
  print("correct: " + str(len(correct)))
  print("wrong: " + str(len(wrong)))
  print("precision: " + str(len(correct)/(len(correct) + len(wrong))))
  return correct, wrong

In [ ]:
## easy_temp_easy_name
ee_correct, ee_wrong = predict_sent(easy_temps, easy_names)

100%|██████████| 15/15 [03:58<00:00, 15.88s/it]


correct: 285
wrong: 15
precision: 0.95


In [ ]:
## easy_temp_hard_name
correct, wrong = predict_sent(easy_temps, hard_names)

100%|██████████| 15/15 [04:05<00:00, 16.36s/it]


correct: 279
wrong: 21
precision: 0.93


In [ ]:
## hard_temp_easy_name
correct, wrong = predict_sent(hard_temps, easy_names)

100%|██████████| 15/15 [07:10<00:00, 28.69s/it]


correct: 263
wrong: 37
precision: 0.8766666666666667


In [ ]:
## hard_temp_hard_name
correct, wrong = predict_sent(hard_temps, hard_names)

100%|██████████| 15/15 [07:07<00:00, 28.51s/it]


correct: 229
wrong: 71
precision: 0.7633333333333333
